In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv("train.csv")
X = df.loc[:,['1stFlrSF', 'BsmtQual', 'ExterQual', 'Fireplaces', 'Foundation',
       'FullBath', 'GarageArea', 'GarageCars', 'GarageFinish',
       'GarageType', 'GarageYrBlt', 'GrLivArea', 'HeatingQC',
       'KitchenQual', 'MSSubClass', 'MasVnrArea', 'MasVnrType',
       'Neighborhood', 'OverallQual', 'TotRmsAbvGrd',
       'TotalBsmtSF', 'YearBuilt', 'YearRemodAdd']].values
y = df.loc[:,["SalePrice"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [22]:
#RFで特徴量の重要度算出。やっぱりOverallQualとGrLivAreaが高い。
rf_reg = RandomForestRegressor(n_estimators = 10)
rf_reg = rf_reg.fit(X_train, y_train)

fti = rf_reg.feature_importances_

fti

C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


array([ 0.03021052,  0.00563764,  0.00893775,  0.00415233,  0.00154622,
        0.00464345,  0.02988605,  0.01897198,  0.00326559,  0.00553993,
        0.01078918,  0.12130107,  0.00143078,  0.00339591,  0.00858215,
        0.03077858,  0.00165797,  0.07251111,  0.5416965 ,  0.00464554,
        0.06444078,  0.01010868,  0.01587029])

In [24]:
X_rf = df.loc[:,["OverallQual", "GrLivArea"]]

In [25]:
X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(X_rf, y, test_size = 0.3, random_state = 0)

In [27]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_rf_train, y_rf_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
# 偏回帰係数を出力
lr.coef_

array([[ 31506.71506056,     61.36857857]])

In [29]:
# 自由度調整済み決定係数
# (決定係数, trainまたはtestのサンプル数, 利用した特徴量の数)
def adjusted(score, n_sample, n_features):
    adjusted_score = 1 - (1 - score) * ((n_sample - 1) / (n_sample - n_features - 1))
    return adjusted_score
# 自由度調整済み決定係数を出力
print('adjusted R^2')
print('train: %3f' % adjusted(lr.score(X_rf_train, y_rf_train), len(y_rf_train), 2))
print('test : %3f' % adjusted(lr.score(X_rf_test, y_rf_test), len(y_rf_test), 2))

adjusted R^2
train: 0.723737
test : 0.688849


In [ ]:
#RMSEを出力する関数を読み込む
from sklearn.metrics import mean_squared_error as mse
# RMSEをtrainとtestに分けて出力
# 過学習をしているかどうかを確認
print('RMSE')
print('train: %.3f' % (mse(y_rf_train, lr.predict(X_train)) ** 1/2))
print('test : %.3f' % (mse(y_rf_test, lr.predict(X_test)) ** 1/2))